### Import Required Libraries and Set Up Environment Variables

In [2]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [3]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [4]:
# Set the base URL
url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json' #"https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:("Movies") AND type_of_material:("Review") AND headline:("love")'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_params = {'api-key': nyt_api_key, 'q': 'python','fq': filter_query, 'sort': sort, 'fl': field_list, 'begin_date': begin_date, 'end_date':end_date}

In [5]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(20):
    # create query with a page number
    query_params = {'api-key': nyt_api_key, 'q': 'python','fq': filter_query, 'sort': sort, 'fl': field_list, 'begin_date': begin_date, 'end_date':end_date,'page':page}
    query_url = url
    # API results show 10 articles at a time    
    
    # Make a "GET" request and retrieve the JSON
    response = requests.get(query_url, params=query_params)
    reviews = response.json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:            
        # loop through the reviews["response"]["docs"] and append each review to the list
        for doc in reviews["response"]["docs"]:
            reviews_list.append(doc)       
            
        # Print the page that was just retrieved
        print(f"Checked page {page + 1}")
        
        # Print the page number that had no results then break from the loop
    except KeyError:
        print(f"Error on {page + 1}")
        break

Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5
Checked page 6
Checked page 7
Checked page 8
Checked page 9
Checked page 10
Checked page 11
Checked page 12
Checked page 13
Checked page 14
Checked page 15
Checked page 16
Checked page 17
Checked page 18
Checked page 19
Checked page 20


In [5]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5],indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2019/10/03/movies/pretenders-review.html",
        "snippet": "Directed by James Franco and featuring James Franco, this film indulges in a distasteful brand of nostalgia.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018Pretenders\u2019 Review: A Little New Wave Love, a Lot of Misogyny",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "Pretenders",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "persons",
                "value": "Franco, James",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "P

In [23]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
df_reviews = pd.json_normalize(reviews_list)
df_reviews

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2019/10/03/movies/pret...,Directed by James Franco and featuring James F...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2019-10-03T11:00:07+0000,276,"‘Pretenders’ Review: A Little New Wave Love, a...",None,None,Pretenders,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
1,https://www.nytimes.com/2016/05/13/movies/love...,"This adaptation, starring Kate Beckinsale, is ...",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2016-05-12T22:02:49+0000,663,"Review: In ‘Love & Friendship,’ Austen Meets W...",None,None,"Pride and Persuasion, Sense and Nonsense, but ...",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None
2,https://www.nytimes.com/2014/09/19/movies/chri...,Christoph Waltz plays a hapless laborer in Ter...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2014-09-18T12:00:07+0000,490,"Work, Love and Therapy, in So Many Bytes",Movie Review,None,"Work, Love and Therapy, in So Many Bytes",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None


In [24]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
df_reviews['title'] = df_reviews['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
df_reviews

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2019/10/03/movies/pret...,Directed by James Franco and featuring James F...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2019-10-03T11:00:07+0000,276,"‘Pretenders’ Review: A Little New Wave Love, a...",None,None,Pretenders,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,Pretenders
1,https://www.nytimes.com/2016/05/13/movies/love...,"This adaptation, starring Kate Beckinsale, is ...",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2016-05-12T22:02:49+0000,663,"Review: In ‘Love & Friendship,’ Austen Meets W...",None,None,"Pride and Persuasion, Sense and Nonsense, but ...",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None,"Love & Friendship,’ Austen Meets Whit Stillma"
2,https://www.nytimes.com/2014/09/19/movies/chri...,Christoph Waltz plays a hapless laborer in Ter...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2014-09-18T12:00:07+0000,490,"Work, Love and Therapy, in So Many Bytes",Movie Review,None,"Work, Love and Therapy, in So Many Bytes",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None,"Work, Love and Therapy, in So Many Byte"


In [26]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
df_reviews['keywords'] = df_reviews['keywords'].apply(extract_keywords)
df_reviews

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2019/10/03/movies/pret...,Directed by James Franco and featuring James F...,The New York Times,"subject: Movies;persons: Franco, James;creativ...",2019-10-03T11:00:07+0000,276,"‘Pretenders’ Review: A Little New Wave Love, a...",None,None,Pretenders,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,Pretenders
1,https://www.nytimes.com/2016/05/13/movies/love...,"This adaptation, starring Kate Beckinsale, is ...",The New York Times,"subject: Movies;persons: Stillman, Whit;person...",2016-05-12T22:02:49+0000,663,"Review: In ‘Love & Friendship,’ Austen Meets W...",None,None,"Pride and Persuasion, Sense and Nonsense, but ...",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None,"Love & Friendship,’ Austen Meets Whit Stillma"
2,https://www.nytimes.com/2014/09/19/movies/chri...,Christoph Waltz plays a hapless laborer in Ter...,The New York Times,"subject: Movies;persons: Gilliam, Terry;person...",2014-09-18T12:00:07+0000,490,"Work, Love and Therapy, in So Many Bytes",Movie Review,None,"Work, Love and Therapy, in So Many Bytes",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None,"Work, Love and Therapy, in So Many Byte"


In [25]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = df_reviews['title'].to_list()
titles

['Pretenders',
 'Love & Friendship,’ Austen Meets Whit Stillma',
 'Work, Love and Therapy, in So Many Byte']

### Access The Movie Database API

In [1]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

NameError: name 'tmdb_api_key' is not defined

In [12]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for title in titles:
    try:
        # Perform a GET request to TMDB search
        tmdb_search_url = url + title.replace(" ", "+") + tmdb_key_string
        tmdb_search_response = requests.get(tmdb_search_url)
        tmdb_search_results = tmdb_search_response.json()

        # Collect movie ID from the first result
        movie_id = tmdb_search_results['results'][0]['id']

        # Make a GET request using the movie ID to retrieve full movie details
        tmdb_movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
        tmdb_movie_response = requests.get(tmdb_movie_url)
        tmdb_movie_details = tmdb_movie_response.json()

        # Extract necessary information
        genres = [genre['name'] for genre in tmdb_movie_details['genres']]
        spoken_languages = [lang['english_name'] for lang in tmdb_movie_details['spoken_languages']]
        production_countries = [country['name'] for country in tmdb_movie_details['production_countries']]

        # Create a dictionary with movie details
        movie_dict = {
            'title': tmdb_movie_details['title'],
            'original_title': tmdb_movie_details['original_title'],
            'budget': tmdb_movie_details['budget'],
            'original_language': tmdb_movie_details['original_language'],
            'homepage': tmdb_movie_details['homepage'],
            'overview': tmdb_movie_details['overview'],
            'popularity': tmdb_movie_details['popularity'],
            'runtime': tmdb_movie_details['runtime'],
            'revenue': tmdb_movie_details['revenue'],
            'release_date': tmdb_movie_details['release_date'],
            'vote_average': tmdb_movie_details['vote_average'],
            'vote_count': tmdb_movie_details['vote_count'],
            'genres': genres,
            'spoken_languages': spoken_languages,
            'production_countries': production_countries
        }

        # Append the dictionary to tmdb_movies_list
        tmdb_movies_list.append(movie_dict)

        # Print out movie name and success message
        print(f"{tmdb_movie_details['title']} found.")
        request_counter += 1

        # Sleep every 50 requests
        if request_counter % 50 == 0:
            time.sleep(1)
            print("Application is sleeping...")

    except IndexError:
        # Print a statement if movie is not found
        print(f"{title} not found.")
        request_counter += 1

Pretenders found.
Love, Divided found.
Work, Love and Therapy, in So Many Byte not found.


In [13]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))


[
    {
        "title": "Pretenders",
        "original_title": "Pretenders",
        "budget": 0,
        "original_language": "en",
        "homepage": "",
        "overview": "A love triangle involving a photographer, a director and an actress.",
        "popularity": 6.168,
        "runtime": 95,
        "revenue": 0,
        "release_date": "2019-09-27",
        "vote_average": 5.6,
        "vote_count": 13,
        "genres": [
            "Drama"
        ],
        "spoken_languages": [
            "English"
        ],
        "production_countries": [
            "United States of America"
        ]
    },
    {
        "title": "Love, Divided",
        "original_title": "Pared con pared",
        "budget": 0,
        "original_language": "es",
        "homepage": "",
        "overview": "She\u2019s a young pianist preparing for an audition. He is a games inventor who can only concentrate in complete silence. Separated by only a paper-thin wall in conjoined apartments, will the

In [14]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)
tmdb_df

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,Pretenders,Pretenders,0,en,,"A love triangle involving a photographer, a di...",6.168,95,0,2019-09-27,5.600,13,[Drama],[English],[United States of America]
1,"Love, Divided",Pared con pared,0,es,,She’s a young pianist preparing for an auditio...,377.803,98,0,2024-04-12,6.519,77,"[Romance, Comedy]",[Spanish],[Spain]


### Merge and Clean the Data for Export

In [15]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(df_reviews, tmdb_df, on='title', how='left')
merged_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2019/10/03/movies/pret...,Directed by James Franco and featuring James F...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2019-10-03T11:00:07+0000,276,"‘Pretenders’ Review: A Little New Wave Love, a...",None,None,Pretenders,...,"A love triangle involving a photographer, a di...",6.168,95.0,0.0,2019-09-27,5.6,13.0,[Drama],[English],[United States of America]
1,https://www.nytimes.com/2016/05/13/movies/love...,"This adaptation, starring Kate Beckinsale, is ...",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2016-05-12T22:02:49+0000,663,"Review: In ‘Love & Friendship,’ Austen Meets W...",None,None,"Pride and Persuasion, Sense and Nonsense, but ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.nytimes.com/2014/09/19/movies/chri...,Christoph Waltz plays a hapless laborer in Ter...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2014-09-18T12:00:07+0000,490,"Work, Love and Therapy, in So Many Bytes",Movie Review,None,"Work, Love and Therapy, in So Many Bytes",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
   # Remove list brackets and quotation marks on the columns containing lists
    # Create a list of the columns that need fixing
columns_to_fix = ['keywords', 'genres', 'spoken_languages', 'production_countries']

    # Create a list of characters to remove
characters_to_remove = ["[", "]", "'", '"']

    # Loop through the list of columns to fix

        # Convert the column to type 'str'
for column in columns_to_fix:
    # Convert the column to type 'str'
        merged_df[column] = merged_df[column].astype(str)

        # Loop through characters to remove
        for char in characters_to_remove:
            merged_df[column] = merged_df[column].str.replace(char, '')

    # Display the fixed DataFrame
print(merged_df)

                                             web_url  \
0  https://www.nytimes.com/2019/10/03/movies/pret...   
1  https://www.nytimes.com/2016/05/13/movies/love...   
2  https://www.nytimes.com/2014/09/19/movies/chri...   

                                             snippet              source  \
0  Directed by James Franco and featuring James F...  The New York Times   
1  This adaptation, starring Kate Beckinsale, is ...  The New York Times   
2  Christoph Waltz plays a hapless laborer in Ter...  The New York Times   

                                            keywords  \
0  {name: subject, value: Movies, rank: 1, major:...   
1  {name: subject, value: Movies, rank: 1, major:...   
2  {name: subject, value: Movies, rank: 1, major:...   

                   pub_date  word_count  \
0  2019-10-03T11:00:07+0000         276   
1  2016-05-12T22:02:49+0000         663   
2  2014-09-18T12:00:07+0000         490   

                                       headline.main headline.kicker  \


C:\Users\sebas\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  app.launch_new_instance()


In [29]:
# Drop "byline.person" column
merged_df.drop(columns=['byline.person'], inplace=True)


NameError: name 'merged_df' is not defined

In [18]:
# Delete duplicate rows and reset index
merged_df.drop_duplicates(inplace=True)
merged_df.reset_index(drop=True, inplace=True)

In [19]:
# Export data to CSV without the index
merged_df.to_csv('merged_data.csv', index=False)